### Build model from planer CNN repo to get segmented images
* Requires Cuda8.0 + gcc5
* To figure out GPU arch, refer this page: https://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/
* For nms & ROI compilation, torch version 0.4.0 is required
* Post compilation upgrade torch to 0.4.1
* Then execute evaluate.py in NVlabs/planercnn on example_images to verify the model

In [1]:
!nvidia-smi
!nvcc --version

Sat Oct 31 07:32:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%shell
echo "Installing CUDA 8"
wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
rm cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub
apt-get update
apt-get install cuda-8-0
pip install torch==0.4.0
sudo apt-get update && \
sudo apt-get install build-essential software-properties-common -y
sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y 
sudo apt-get update
sudo apt-get install gcc-5 g++-5 -y
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 50 --slave /usr/bin/g++ g++ /usr/bin/g++-5 
sudo apt-get install gcc-5 g++-5 g++-5-multilib gfortran-5
sudo update-alternatives --config gcc

echo "Downloading planercnn"

git clone https://github.com/NVlabs/planercnn
cd planercnn/nms/src/cuda/
echo "Compiling nms"
nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_60
cd ../../
python build.py
cd ..
cd roialign/roi_align/src/cuda/
echo "Compiling roi_align"
nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_60
cd ../../
python build.py
cd ../../
pip install torch==0.4.1
wget https://www.dropbox.com/s/yjcg6s57n581sk0/checkpoint.zip?dl=0
mkdir checkpoint
mv "checkpoint.zip?dl=0" "planercnn_refine.zip"
mv planercnn_refine.zip checkpoint/
cd checkpoint/
unzip planercnn_refine.zip
rm planercnn_refine.zip
cd ..
echo "Running Tests"
python evaluate.py --methods=f --suffix=warping_refine --dataset=inference --customDataFolder=example_images
echo "Setup Complete"

Installing CUDA 8
--2020-10-31 07:32:55--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?KNGAe0_UtguuFFIMlMb8zhbRP4_PEQSDosRDRuXjG0Ssys_wOZolPlc_bg6xst_dHkI16a2llLn3LAMXgsndE1xbtLqHKV6LuoRGc6_MSbzRdJtspOb1RusxpnJYMb7YwStZ0MK7p6pk6KRrZEnu-WoTWVD-yDsJk3eFw_C66xTbBltbq3m6qXiyDd8RTXQKCfwHK_YW0r1hvsmU22LRserbmg [following]
--2020-10-31 07:32:55--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?KNGAe0_UtguuFFIMlMb8zhbRP4_PEQSDosRDRuXjG0Ssys_wOZolPlc_bg6xst_dHkI16a2llLn3LAMX

### Mount drive to fetch input dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/My\ Drive/EVA/Datasets/YoloV3-PPE/YoloV3_Dataset.zip
!mv YoloV3_Dataset planercnn

### Execute evaluation script to get segmented images
* Note: It will generate planer images & depth images as well. Do not store them, as we will be using different model for that
* Camera calibration can be done as mentioned here: https://www.learnopencv.com/camera-calibration-using-opencv/
* But in our case, we'll use the default camera.txt file from repo's example_images

In [11]:
%%shell
cd planercnn
touch test/inference/dummy_final.png
rm YoloV3_Dataset/Images/*.txt
cp example_images/camera.txt YoloV3_Dataset/Images/
python evaluate.py --methods=f --suffix=warping_refine --dataset=inference --customDataFolder=YoloV3_Dataset/Images --numTestingImages=3546

rm: cannot remove 'YoloV3_Dataset/Images/*.txt': No such file or directory
the number of images 3545
  0% 0/3545 [00:00<?, ?it/s]/content/planercnn/models/model.py:1473: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)
THCudaCheck FAIL file=/pytorch/aten/src/THC/THCGeneral.cpp line=663 error=11 : invalid argument
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior wh

### Save the segmented images to drive

In [12]:
%%shell
cd planercnn
rm test/inference/*depth*
cp test/inference/* /content/drive/My\ Drive/EVA/Datasets/YoloV3-PPE/PlanerCNN-Segmentation/

In [13]:
!du -sh /content/drive/My\ Drive/EVA/Datasets/YoloV3-PPE/PlanerCNN-Segmentation

6.0G	/content/drive/My Drive/EVA/Datasets/YoloV3-PPE/PlanerCNN-Segmentation
